In [1]:
# !wget https://github.com/mesolitica/llama2-embedding/raw/main/test-set/twitter-dataset-bge-test.sample.json
# !wget https://huggingface.co/datasets/mesolitica/embedding-pair-mining/resolve/main/twitter-dataset-bge-test.jsonl

In [2]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE pip3 install flash-attn --no-build-isolation
# !pip3 install transformers -U

In [3]:
!nvidia-smi

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Thu Feb 15 10:24:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000001:00:00.0 Off |                    0 |
| N/A   41C    P0              66W / 300W |   3677MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True, device = 'cuda')

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

loading existing colbert_linear and sparse_linear---------


In [5]:
import json
import openai
import torch
from tqdm import tqdm

In [6]:
with open('c.cari.com.my-dataset-bge-test.sample.json') as fopen:
    rev_data = json.load(fopen)

In [7]:
strings = rev_data.keys()

In [8]:
embeddings_1 = model.encode(strings, 
                            batch_size=12, 
                            max_length=8192,
                            )['dense_vecs']

encoding: 100%|██████████| 1296/1296 [00:40<00:00, 32.01it/s]


In [9]:
vectors = {}
for no, s in enumerate(strings):
    vectors[s] = embeddings_1[no]

In [10]:
no_string = {no: k for no, (k, v) in enumerate(vectors.items())}
string_no = {v: k for k, v in no_string.items()}
no_string[0]

'回复 5# 当猫爱上狗  我是27号要用的， 我想应该是来不及了。。真的是很谢谢你的回复。。'

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [12]:
np_vectors = np.array(list(vectors.values()))
np_vectors.shape

(15545, 1024)

In [13]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}
total = 0
with open('c.cari.com.my-dataset-bge-test.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        query = l['query'].strip()
        query_no = string_no.get(query)
        if query_no is None:
            continue
        for s in l['pos']:
            s = s.strip()
            v_s = vectors.get(s)
            s_no = string_no.get(s)
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            argsort = np.argsort(cosine_similarity(v_s, np_vectors)[0])[::-1]
            for k in tops.keys():
                if s_no in argsort[:k]:
                    k_ = k + 1
                else:
                    k_ = k
                if query_no in argsort[:k_]:
                    tops[k] += 1
            total += 1
            
for k, v in tops.items():
    print(k, v / total)

7087it [18:01,  6.55it/s] 

1 0.08795478629459555
3 0.23754856941010244
5 0.33574708583539387
10 0.4558459908159661
